## Imports

In [1]:
from datetime import datetime
import gmpy2


## Read info from files

In [2]:
se = 'SE_12.txt'
mitm = 'MitM_12.txt'

se_C_data = []
se_N_data = []
mitm_data = []

with open(se, 'r') as f:
    lines = f.readlines()

    for index in range(0, len(lines), 2):
        se_C_data.append(int(lines[index].split('=')[1].strip(), base=16))
        se_N_data.append(int(lines[index + 1].split('=')[1].strip(), base=16))

with open(mitm, 'r') as f:
    lines = f.readlines()
    mitm_data.append(int(lines[0].split('=')[1].strip(), base=16))
    mitm_data.append(int(lines[1].split('=')[1].strip(), base=16))


## Chinese Remainder Theorem Attack

In [3]:
E = 5

def gcdExtended(a, b):
    if a == 0 :
        return b, 0, 1

    gcd, u1, v1 = gcdExtended(b % a, a)

    u = v1 - (b // a) * u1
    v = u1

    return gcd, u, v

def CRT(arr_C, arr_N):
	if len(arr_C) != len(arr_N):
		return None

	main_modulo = 1
	for elem in arr_N:
		main_modulo *= elem

	modules = [main_modulo // elem for elem in arr_N]
	answer = 0

	for index in range(len(arr_C)):
		gcd, u, v = gcdExtended(arr_N[index], modules[index])

		answer += v * modules[index] * arr_C[index]

	return answer % main_modulo


In [11]:
start_time = datetime.now()
big_C = CRT(se_C_data, se_N_data)
end_time = datetime.now() - start_time

message = gmpy2.iroot(big_C, E)

print(hex(message[0]))
print(message[0] ** E == big_C)
print(f'{end_time.total_seconds() * 1000}ms')


0x1ffffffffffffffff00cd2321855b7a635d71b995fdc7d2a7ad454fedd43e6fe954d9728fa467a6ca5add883250a45dcf69944ef804eb3d990abe981bda330d6c3e5740f219963cc6bf42d1cd209ef430e86988a9955c82862857e0271a01cdf9d962b3639daa47b531c1e22ac1c6e4f0bc069c6a0b8176eb1cdd1496a5bc
True
2.0010000000000003ms


## Meet in the Middle Attack

In [5]:
e = 65537
l = 20
c, n = mitm_data


In [6]:
def meet_in_the_middle(c, e, n, l):
	t_array = []

	current_range = [*range(0, 1 << (l >> 1))]

	for i in current_range:
		t_array.append(gmpy2.powmod(i + 1, e, n))

	for i in current_range:
		c_s = gmpy2.f_mod(gmpy2.mul(c, gmpy2.invert(t_array[i], n)), n)

		for j in current_range:
			if c_s == t_array[j]:
				return (i + 1, j + 1)

	return None


In [12]:
start_time = datetime.now()
result = meet_in_the_middle(c, e, n, l)
end_time = datetime.now() - start_time

if result != None:
	t, s = result
	print(hex(t * s % n))

	print(gmpy2.powmod(t * s, e, n) == c)
else:
	print(':(')

print(f'{end_time.total_seconds() * 1000}ms')


0x85ef5
True
53.324999999999996ms


## Brute Force Attack

In [8]:
def brute_force_attack(c, e, n, l):
	for i in range(n):
		if (gmpy2.powmod(i, e, n) == c):
			return i


In [9]:
start_time = datetime.now()
result = brute_force_attack(c, e, n, l)
end_time = datetime.now() - start_time

print(hex(result))
print(f'{end_time.total_seconds() * 1000}ms')


548597
10268.13ms
